Connected to Python 3.13.7

# Libraries

In [60]:
from pyomo.environ import Set, Param, Var, NonNegativeReals, ConcreteModel, SolverFactory, Objective, Constraint, minimize, Binary
import pandas as pd
import numpy as np
import json
import random

# Get Data

In [61]:
distance_matrix = pd.read_csv("../processed_data/distance_matrix.csv", header=None).to_numpy()
duration_matrix = pd.read_csv("../processed_data/duration_matrix.csv", header=None).to_numpy()
some_stations = pd.read_csv("../processed_data/some_stations.csv")

In [62]:
distance_matrix

array([[ 0.    , 11.4205,  8.4061, ...,  2.2988, 19.6956, 16.4128],
       [10.2816,  0.    ,  5.8469, ...,  9.098 , 17.1364,  6.3585],
       [ 9.1415,  5.1527,  0.    , ...,  7.9579, 10.9358,  7.963 ],
       ...,
       [ 2.2852,  9.8422,  6.8125, ...,  0.    , 18.102 , 14.8192],
       [20.4347, 12.964 , 11.7789, ..., 19.2512,  0.    ,  8.8207],
       [18.8965,  5.6976, 10.0991, ..., 17.713 ,  8.9202,  0.    ]],
      shape=(100, 100))

In [63]:
duration_matrix

array([[ 0.       , 14.761666 , 10.006667 , ...,  4.778333 , 22.813334 ,
        23.6      ],
       [11.818334 ,  0.       ,  8.191667 , ..., 11.906667 , 20.998333 ,
        13.405    ],
       [12.04     , 11.136666 ,  0.       , ..., 12.128333 , 15.728333 ,
        16.738333 ],
       ...,
       [ 5.5616665, 15.275    , 10.611667 , ...,  0.       , 23.418333 ,
        24.205    ],
       [22.381666 , 20.615    , 15.57     , ..., 22.47     ,  0.       ,
        11.92     ],
       [23.861666 , 12.065    , 16.273333 , ..., 23.95     , 12.183333 ,
         0.       ]], shape=(100, 100))

In [64]:
some_stations.head()

,station_name,station_id,latitude,longitude,net_flow
0,68 St & 5 Ave,2698.07,40.634160,-74.020580,0
1,West St & Liberty St,5184.08,40.711444,-74.014847,6
2,Montague St & Hicks St,4718.11,40.695224,-73.995989,-6
3,20 Ave & Shore Blvd,7391.02,40.785994,-73.915098,0
4,Frederick Douglass Blvd & Harlem River Dr,8100.01,40.830702,-73.936371,0


In [65]:
print(sum(some_stations['net_flow']))

0


# Build Model

In [66]:
model = ConcreteModel()
solver = SolverFactory('glpk')

N_BUSES = 5
MIN_BUS_CAPACITY = 10
MAX_BUS_CAPACITY = 30

M = 1e6

## Build the Sets

In [67]:
model.del_component('nodes')
model.del_component('source_stations')
model.del_component('sink_stations')
model.del_component('buses')
model.del_component('visit_nodes')

model.nodes = Set(initialize=some_stations['station_name'].tolist())
model.source_stations = Set(initialize=some_stations[some_stations['net_flow'] > 0]['station_name'].tolist())
model.sink_stations = Set(initialize=some_stations[some_stations['net_flow'] < 0]['station_name'].tolist())

model.visit_nodes = model.source_stations.union(model.sink_stations)
model.all_nodes = model.visit_nodes.union(Set(initialize=['Warehouse']))
# Print whether 'Warehouse' is in visit_nodes and its index in some_stations

if 'Warehouse' in model.visit_nodes:
    warehouse_idx = some_stations[some_stations['station_name'] == 'Warehouse'].index[0]
    print(f"'Warehouse' is in visit_nodes at index {warehouse_idx} in some_stations.")
else:
    print("'Warehouse' is not in visit_nodes.")

print(f"Total nodes to visit: {len(model.visit_nodes)}")
print(f"Visit Nodes: {model.visit_nodes.data()}")

model.buses = Set(initialize=[f"bus_{i}" for i in range(N_BUSES)])

'Warehouse' is not in visit_nodes.
Total nodes to visit: 64
Visit Nodes: ('West St & Liberty St', 'E 115 St & Madison Ave', '108 St & 39 Ave', 'Boston Rd & E 178 St', 'Willis Ave & E 141 St', 'E 161 St & River Ave', 'Gramercy Park N & Gramercy Park E', 'E 33 St & 1 Ave', '75 St & Northern Blvd', 'W 4 St & 7 Ave S', '52 St & 1 Ave', 'Driggs Ave & N Henry St', 'Metropolitan Ave & Bedford Ave', 'Broadway & Morris St', 'Caton Ave & St. Pauls Pl', 'Market St & Cherry St', 'Broadway & 31 St', 'Hudson St & W 13 St', '30 Ave & 12 St', 'Broadway & W 160 St', 'E 191 St & Bathgate Ave', 'W 22 St & 10 Ave', 'E 103 St & 2 Ave', '3 Ave & E 82 St', 'Central Park W & W 68 St', 'Schermerhorn St & Court St', 'E 167 St & Franklin Ave', 'Dock St & Front St', 'E 163 St & Union Ave', 'Berkeley Pl & 7 Ave', 'Brooklyn Ave & Dean St', 'E 2 St & 2 Ave', 'Montague St & Hicks St', 'Crotona Ave & E 183 St', '36 St & 4 Ave', '37 Ave & 35 St', 'W 31 St & 7 Ave', 'Smith St & 3 St', 'Fort Hamilton Pkwy & E 5 St', 'Tin

## Bus Parameters

In [68]:
bus_param_dict = {
    f"bus_{i}": {
        random.randint(MIN_BUS_CAPACITY, MAX_BUS_CAPACITY)
    } for i in range(N_BUSES)
}

model.del_component('busCapacities')

# Flatten the bus_param_dict so that keys are bus names and values are capacities (not sets)
bus_param_flat = {k: list(v)[0] for k, v in bus_param_dict.items()}

model.busCapacities = Param(model.buses, initialize=bus_param_flat, within=NonNegativeReals)


# Node Parameters

In [69]:
# We need to build the parameter dictionaries for each station which contain their supply and demand 
""" supply_param_dict = name : [supply, demand]"""

model.del_component('supply')
model.del_component('demand')

supply_param_dict = {
    row['station_name']: [
        row['net_flow'] if row['net_flow'] > 0 else 0,
        -row['net_flow'] if row['net_flow'] < 0 else 0
    ] for _, row in some_stations.iterrows()
}

model.supply = Param(
    model.visit_nodes,
    initialize={k: v[0] for k, v in supply_param_dict.items() if k in model.visit_nodes},
    within=NonNegativeReals,
    mutable=True
)

model.demand = Param(
    model.visit_nodes,
    initialize={k: v[1] for k, v in supply_param_dict.items() if k in model.visit_nodes},
    within=NonNegativeReals,
    mutable=True
)


# Decision Variables 

In [70]:
model.del_component('x')

# We need to decide if a bus k goes from node i to node j
# x_kij
model.x = Var(
    model.buses,
    model.all_nodes,
    model.all_nodes,
    within=Binary
)

# Misc Varaibles

In [71]:
# We need to model how many bikes are on board each bus k when it arrives at node i
model.del_component('on_board')

model.on_board = Var(
    model.all_nodes,
    model.buses,
    within=NonNegativeReals,
)

## Objective Function

In [ ]:
def min_distance_obj(model):
    return sum(
        distance_matrix[
            some_stations[some_stations['station_name'] == i].index[0],
            some_stations[some_stations['station_name'] == j].index[0]
        ] * model.x[k, i, j]
        for k in model.buses
        for i in model.all_nodes
        for j in model.all_nodes
        if i != j
    )

def min_time_obj(model):
    return sum(
        duration_matrix[
            some_stations[some_stations['station_name'] == i].index[0],
            some_stations[some_stations['station_name'] == j].index[0]
        ] * model.x[k, i, j]
        for k in model.buses
        for i in model.all_nodes
        for j in model.all_nodes
        if i != j
    )

model.del_component('objective')

model.objective = Objective(rule=min_distance_obj, sense=minimize)



## CONSTRAINTS

In [ ]:
""" FLOW CONSERVATION CONSTRAINTS """
def flow_conservation_rule(model, n, k):
    inflow = sum(
        model.x[k, i, n]
        for i in model.all_nodes
        if i != n
    )
    
    outflow = sum(
        model.x[k, n, j]
        for j in model.all_nodes
        if j != n
    )
    
    return inflow == outflow


In [ ]:
""" ASSIGNMENT CONSTRAINTS """
def assignment_rule(model, n):
    if n == 'Warehouse':
        return Constraint.Skip
    
    return sum(
        model.x[k, n, j]
        for k in model.buses
        for j in model.all_nodes
        if j != n
    ) == 1

In [ ]:
""" VEHICLE LOADING CONSTRAINT """
# Updates load for trips going to any node EXCEPT the Warehouse
def vehicle_loading_rule(model, i, j, k):
    # If we are returning to the warehouse, SKIP this rule.
    # We don't want to force on_board['Warehouse'] to equal the return load.
    if j == 'Warehouse':
        return Constraint.Skip

    # Calculate flow at node i
    if i == 'Warehouse':
        flow_i = 0 # No flow change at depot, just carrying what we started with
    else:
        flow_i = model.supply[i] if i in model.source_stations else -model.demand[i]

    # Standard MTZ constraint
    return model.on_board[j, k] >= model.on_board[i, k] + flow_i - M*(1-model.x[k, i, j])

In [ ]:
""" RETURN TRIP CAPACITY CONSTRAINT """
# Ensures that when returning to the warehouse, the bus isn't overloaded
def return_capacity_rule(model, i, k):
    # We only care about trips ending at the Warehouse
    if i == 'Warehouse':
        return Constraint.Skip
        
    flow_i = model.supply[i] if i in model.source_stations else -model.demand[i]
    
    # If x[k, i, 'Warehouse'] == 1, then Load_i + Flow_i <= Capacity
    return model.on_board[i, k] + flow_i <= model.busCapacities[k] + M*(1-model.x[k, i, 'Warehouse'])

""" RETURN TRIP NON-NEGATIVE CONSTRAINT """
# Ensures that when returning to the warehouse, the bus has >= 0 bikes
def return_non_negative_rule(model, i, k):
    # We only care about trips ending at the Warehouse
    if i == 'Warehouse':
        return Constraint.Skip
        
    flow_i = model.supply[i] if i in model.source_stations else -model.demand[i]
    
    # If x[k, i, 'Warehouse'] == 1, then Load_i + Flow_i >= 0
    return model.on_board[i, k] + flow_i >= 0 - M*(1-model.x[k, i, 'Warehouse'])

In [ ]:
""" VEHICLE CAPACITY CONSTRAINT """
def vehicle_capacity_rule(model, i, k):
    return model.on_board[i, k] <= model.busCapacities[k]


In [ ]:
""" SUBTOUR ELIMINATION CONSTRAINTS """
def subtour_elimination_rule(model, i, j, k):
    if i != j:
        return model.x[k,i,j] <= model.S.__len__() - 1
    else:
        return Constraint.Skip

In [ ]:
""" DEPOT CONSTRAINTS """
def leave_depot_rule(model, k):
    return sum(
        model.x[k, 'Warehouse', j]
        for j in model.visit_nodes
    ) >= 1

def return_to_depot_rule(model, k):
    return sum(
        model.x[k, i, 'Warehouse']
        for i in model.visit_nodes
    ) >= 1

# Building and Solving

In [ ]:

model.del_component('flow_conservation')
model.del_component('assignment')
model.del_component('loading')
model.del_component('return_capacity')     
model.del_component('return_non_negative') 
model.del_component('capacity')
model.del_component('start_depot')
model.del_component('end_depot')


model.flow_conservation = Constraint(
    model.visit_nodes,
    model.buses,
    rule=flow_conservation_rule
)

model.assignment = Constraint(
    model.visit_nodes,
    rule=assignment_rule
)

model.loading = Constraint(
    model.all_nodes, # i
    model.all_nodes, # j
    model.buses,     # k
    rule=vehicle_loading_rule
)

model.return_capacity = Constraint(
    model.visit_nodes, # i (only need to check stations returning to depot)
    model.buses,       # k
    rule=return_capacity_rule
)

model.return_non_negative = Constraint(
    model.visit_nodes, # i
    model.buses,       # k
    rule=return_non_negative_rule
)

model.capacity = Constraint(
    model.all_nodes,
    model.buses,
    rule=vehicle_capacity_rule
)

model.start_depot = Constraint(
    model.buses,
    rule=leave_depot_rule
)

model.end_depot = Constraint(
    model.buses,
    rule=return_to_depot_rule
)

solver.options['tmlim'] = 300  # Time limit in seconds (e.g., 300s = 5 mins)
solver.options['mipgap'] = 0.05 # Stop if gap is less than 5%

sol = solver.solve(model, tee=True)



ERROR: Rule failed when generating expression for Constraint loading with
index ('Warehouse', 'West St & Liberty St', 'bus_0'): KeyError: "Index
'('Warehouse', 'bus_0')' is not valid for indexed component 'on_board'"
ERROR: Constructing component 'loading' from data=None failed:
        KeyError: "Index '('Warehouse', 'bus_0')' is not valid for indexed
        component 'on_board'"


KeyError: "Index '('Warehouse', 'bus_0')' is not valid for indexed component 'on_board'"

In [ ]:
print("Print values for each variable explicitly")
for i in model.x:
    if model.x[i].value is not None and model.x[i].value > 0:
        print(str(model.x[i]), model.x[i].value)

# for i in model.y:
#   print(str(model.y[i]), model.y[i].value)
# print("")

Print values for each variable explicitly
x[bus_0,108 St & 39 Ave,34 Ave & 108 St] 1.0
x[bus_0,E 33 St & 1 Ave,Warehouse] 1.0
x[bus_0,75 St & Northern Blvd,62 St & Northern Blvd] 1.0
x[bus_0,34 Ave & 108 St,108 St & 39 Ave] 1.0
x[bus_0,62 St & Northern Blvd,75 St & Northern Blvd] 1.0
x[bus_0,Warehouse,E 33 St & 1 Ave] 1.0
x[bus_1,Gramercy Park N & Gramercy Park E,Warehouse] 1.0
x[bus_1,52 St & 1 Ave,36 St & 4 Ave] 1.0
x[bus_1,Driggs Ave & N Henry St,Graham Ave & Withers St] 1.0
x[bus_1,'Caton Ave & St. Pauls Pl',Fort Hamilton Pkwy & E 5 St] 1.0
x[bus_1,Broadway & 31 St,37 Ave & 35 St] 1.0
x[bus_1,30 Ave & 12 St,24 Ave & 26 St] 1.0
x[bus_1,Central Park W & W 68 St,E 68 St & 3 Ave] 1.0
x[bus_1,Schermerhorn St & Court St,Smith St & 3 St] 1.0
x[bus_1,Dock St & Front St,Schermerhorn St & Court St] 1.0
x[bus_1,Berkeley Pl & 7 Ave,Montague St & Hicks St] 1.0
x[bus_1,Brooklyn Ave & Dean St,Monroe St & Marcus Garvey Blvd] 1.0
x[bus_1,Montague St & Hicks St,Dock St & Front St] 1.0
x[bus_1,36 St 

# Save Results

In [ ]:
def save_results(save_path = "../results/solutionv2.csv", model = None):
    rows = []
    for key in model.x.keys():
        bus, i, j = key
        desc = model.x[key].value
        rows.append({
            "bus": bus,
            "from": i,
            "to": j,
            "value": desc
        })

    df = pd.DataFrame(rows)
    df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

save_results(model=model)

Results saved to ../results/solution.csv


# View Results

In [ ]:
def view_results(model,df ,  bus:int=0):
    # View Results
    print("Objective Value: ", model.objective())
    print(f"The route taken by bus_{bus}:\n")
    df_bus = df[df['bus'] == f'bus_{bus}']
    
    route = ['Warehouse']
    current_node = df_bus[(df_bus['from'] == 'Warehouse') & (df_bus['value'] == 1)]['to'].values[0]
    route.append(current_node)
    
    while current_node != 'Warehouse':
        next_node = df_bus[(df_bus['from'] == current_node) & (df_bus['value'] == 1)]['to'].values
        if len(next_node) == 0:
            break
        current_node = next_node[0]
        route.append(current_node)
    
    print(route)

view_results(model, pd.read_csv("../results/solution.csv"), bus=0)
view_results(model, pd.read_csv("../results/solution.csv"), bus=1)
view_results(model, pd.read_csv("../results/solution.csv"), bus=2)
view_results(model, pd.read_csv("../results/solution.csv"), bus=3)
view_results(model, pd.read_csv("../results/solution.csv"), bus=4)


Objective Value:  130.58219999999997
The route taken by bus_0:

['Warehouse', 'E 33 St & 1 Ave', 'Warehouse']
Objective Value:  130.58219999999997
The route taken by bus_1:

['Warehouse', 'Gramercy Park N & Gramercy Park E', 'Warehouse']
Objective Value:  130.58219999999997
The route taken by bus_2:

['Warehouse', 'W 22 St & 10 Ave', 'Warehouse']
Objective Value:  130.58219999999997
The route taken by bus_3:

['Warehouse', 'W 31 St & 7 Ave', 'Warehouse']
Objective Value:  130.58219999999997
The route taken by bus_4:

['Warehouse', 'Greenwich Ave & 8 Ave', 'Hudson St & W 13 St', 'Warehouse']


# Misc Functions

In [ ]:
def form_pair_set(nodes):
    pairs = []
    for i in nodes:
        for j in nodes:
            if i != j:
                pairs.append((i, j))

    return Set(initialize=form_pair_set(model.visit_nodes))
